In [ ]:
import firebase_admin
from firebase_admin import credentials, db
import csv
import time
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

if not firebase_admin._apps:
    cred = credentials.Certificate('/content/drive/MyDrive/CaneConnect/JSON file.json')
    firebase_admin.initialize_app(cred, {'databaseURL': 'https://caneconnect-a0ed2-default-rtdb.asia-southeast1.firebasedatabase.app/'})
print("Firebase initialized:", firebase_admin._apps)

def get_sensor_data():
    ref = db.reference('/Sensor')
    data = ref.get()
    print("Full Database Snapshot:", data)
    return data if isinstance(data, dict) else None

csv_file_path = "/content/drive/MyDrive/Fallevents_data.csv"

try:
    with open(csv_file_path, mode='x', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["AccX", "AccY", "AccZ", "RotX", "RotY", "RotZ"])
except FileExistsError:
    pass

while True:
    data = get_sensor_data()
    print("Fetched data:", data)

    if data and isinstance(data, dict):
        try:
            with open(csv_file_path, mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([
                    data.get("accX", "N/A"),
                    data.get("accY", "N/A"),
                    data.get("accZ", "N/A"),
                    data.get("rotX", "N/A"),
                    data.get("rotY", "N/A"),
                    data.get("rotZ", "N/A")
                ])
            print("Data saved to CSV:", data)
        except Exception as e:
            print("Error writing to CSV:", e)

    time.sleep(1)
